# Import

In [17]:
# Gym
import gymnasium as gym
from gymnasium import Env
from gymnasium.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

# helpers
import numpy as np
import random
import os

# Stable-baseline
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

from typing import Any

# Types of Spaces

In [18]:
Discrete(3).sample()

np.int64(2)

In [19]:
Box(0, 1, shape=(3,3)).sample()

array([[0.06297716, 0.10702176, 0.01973697],
       [0.6366031 , 0.24294029, 0.8288127 ],
       [0.9864764 , 0.37008932, 0.18712899]], dtype=float32)

In [20]:
Tuple((Discrete(3), Box(0, 1, shape=(3,3)))).sample()

(np.int64(0),
 array([[0.13601941, 0.38872436, 0.94415855],
        [0.5275697 , 0.25295538, 0.23016934],
        [0.15140302, 0.23499121, 0.4273569 ]], dtype=float32))

In [21]:
Dict({'height' : Discrete(3), 'place' : Box(0,1, shape=(1,2)), "tuple" : Tuple((Discrete(3), Box(0, 1, shape=(3,3))))}).sample()

{'height': np.int64(2),
 'place': array([[0.41047266, 0.4029648 ]], dtype=float32),
 'tuple': (np.int64(2),
  array([[0.18487631, 0.40628025, 0.48229235],
         [0.6910071 , 0.34391835, 0.50859517],
         [0.25931424, 0.02871772, 0.9822588 ]], dtype=float32))}

In [22]:
MultiBinary((4,2)).sample()

array([[1, 1],
       [0, 1],
       [1, 1],
       [1, 1]], dtype=int8)

In [23]:
MultiDiscrete([2,3,5]).sample()

array([0, 1, 3])

# Building Environment

In [24]:
Box(low=np.array([0]), high=np.array([100])).sample()

array([45.474876], dtype=float32)

In [25]:
Box(low=0, high=100).sample()

array([82.41609], dtype=float32)

In [26]:
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3, start=-1)
        self.observation_space = Box(low=0, high=100)
        self.state = float(38 + random.randint(-3,3))
        self.shower_length = 60

    def step(self, action):
        print(f"🔍 DEBUG: Action received: {action}")
        self.state += action

        self.shower_length -= 1

        MIN_TEMP = 37
        MAX_TEMP = 39
        reward = 1 if self.state >= MIN_TEMP and self.state <= MAX_TEMP else -1

        reward = -abs(self.state - 38)

        if self.shower_length <= 0:
            terminated = True
            truncated = True
        else:
            terminated = False
            truncated = False


        info = {}

        return np.array([self.state], dtype=np.float32), reward, terminated, truncated, info

    def render(self):
        pass

    def reset(
        self,
        *,
        seed: int | None = None,
        options: dict[str, Any] | None = None,
    ):
        self.state = float(38 + random.randint(-3,3))
        self.shower_length = 60
        info = {}

        return self.state, info

In [27]:
env = ShowerEnv()

In [28]:
env.observation_space.sample()

array([44.90915], dtype=float32)

In [29]:
env.action_space.sample()

np.int64(0)

# Test Environment

In [30]:
episodes = 10
for episode in range(1, episodes+1):
    env.reset()
    terminated = False
    truncated = False
    score = 0

    while not terminated and not truncated:
        action = env.action_space.sample()
        obs, reward, terminated, truncated , info = env.step(action)
        score += reward
    print(f"Episode: {episode}  Score {score}")
env.close()

🔍 DEBUG: Action received: 1
🔍 DEBUG: Action received: -1
🔍 DEBUG: Action received: -1
🔍 DEBUG: Action received: 0
🔍 DEBUG: Action received: 0
🔍 DEBUG: Action received: 1
🔍 DEBUG: Action received: 0
🔍 DEBUG: Action received: 1
🔍 DEBUG: Action received: 0
🔍 DEBUG: Action received: 1
🔍 DEBUG: Action received: 1
🔍 DEBUG: Action received: 1
🔍 DEBUG: Action received: 0
🔍 DEBUG: Action received: 1
🔍 DEBUG: Action received: 1
🔍 DEBUG: Action received: 0
🔍 DEBUG: Action received: -1
🔍 DEBUG: Action received: 0
🔍 DEBUG: Action received: -1
🔍 DEBUG: Action received: 1
🔍 DEBUG: Action received: 0
🔍 DEBUG: Action received: 1
🔍 DEBUG: Action received: -1
🔍 DEBUG: Action received: -1
🔍 DEBUG: Action received: 1
🔍 DEBUG: Action received: 1
🔍 DEBUG: Action received: 1
🔍 DEBUG: Action received: 1
🔍 DEBUG: Action received: 0
🔍 DEBUG: Action received: 0
🔍 DEBUG: Action received: 1
🔍 DEBUG: Action received: 0
🔍 DEBUG: Action received: 0
🔍 DEBUG: Action received: 0
🔍 DEBUG: Action received: -1
🔍 DEBUG: Acti

# Train

In [31]:
log_path = os.path.join('training','logs')
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [32]:
model.learn(800000, reset_num_timesteps=False)

Logging to training/logs/PPO_17
🔍 DEBUG: Action received: 1
🔍 DEBUG: Action received: 2
🔍 DEBUG: Action received: 0
🔍 DEBUG: Action received: 0
🔍 DEBUG: Action received: 2
🔍 DEBUG: Action received: 2
🔍 DEBUG: Action received: 0
🔍 DEBUG: Action received: 0
🔍 DEBUG: Action received: 2
🔍 DEBUG: Action received: 0
🔍 DEBUG: Action received: 1
🔍 DEBUG: Action received: 1
🔍 DEBUG: Action received: 1
🔍 DEBUG: Action received: 2
🔍 DEBUG: Action received: 2
🔍 DEBUG: Action received: 0
🔍 DEBUG: Action received: 2
🔍 DEBUG: Action received: 0
🔍 DEBUG: Action received: 2
🔍 DEBUG: Action received: 2
🔍 DEBUG: Action received: 1
🔍 DEBUG: Action received: 2
🔍 DEBUG: Action received: 2
🔍 DEBUG: Action received: 0
🔍 DEBUG: Action received: 2
🔍 DEBUG: Action received: 0
🔍 DEBUG: Action received: 0
🔍 DEBUG: Action received: 1
🔍 DEBUG: Action received: 0
🔍 DEBUG: Action received: 1
🔍 DEBUG: Action received: 1
🔍 DEBUG: Action received: 2
🔍 DEBUG: Action received: 1
🔍 DEBUG: Action received: 0
🔍 DEBUG: Action 

KeyboardInterrupt: 

# Save

In [ ]:
shower_path = os.path.join('training','model','Shower_Model_PPO')

In [ ]:
model.save(shower_path)

/Users/fauzanghaza/Applications/miniconda3/envs/research/lib/python3.10/site-packages/stable_baselines3/common/save_util.py:284: UserWarning: Path 'training/model' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [ ]:
del model

In [ ]:
model = PPO.load(shower_path, env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
evaluate_policy(model, env, 10)

(np.float64(-58.2), np.float64(1.0770329614269007))